<a href="https://colab.research.google.com/github/Tecindes/ChallengeONE__/blob/main/TelecomX_parte2_latam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparación de los datos

## Extracción del archivo tratado

In [80]:
import pandas as pd
# URL raw del archivo CSV (ajusta con tu repo y path)
url_csv = 'https://raw.githubusercontent.com/Tecindes/ChallengeONE__/refs/heads/main/reporte.csv'


# Cargar el CSV directamente como DataFrame
df = pd.read_csv(url_csv)

# Vista previa
df.sample(5)

,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,...,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total,SeniorLabel,ChurnLabel
1447,2137-DQMEV,No,Male,0,Yes,Yes,68,No,No phone service,DSL,...,No,Yes,No,One year,No,Mailed check,44.8,2983.65,No,0
515,0749-IRGQE,No,Female,1,Yes,No,13,No,No phone service,DSL,...,No,Yes,Yes,Month-to-month,No,Electronic check,45.3,528.45,Sí,0
5137,7233-PAHHL,No,Male,0,Yes,Yes,66,Yes,Yes,DSL,...,Yes,Yes,Yes,Two year,Yes,Mailed check,84.0,5714.25,No,0
2448,3508-CFVZL,No,Female,0,No,No,71,Yes,Yes,Fiber optic,...,Yes,Yes,Yes,Two year,No,Mailed check,111.3,7985.90,No,0
1051,1536-HBSWP,No,Female,0,No,No,1,Yes,No,DSL,...,Yes,No,No,Month-to-month,Yes,Electronic check,49.9,49.90,No,0


## Eliminación de columnas irrelevantes

In [90]:
df.columns


Index(['Churn', 'customer.gender', 'customer.Partner', 'customer.Dependents',
       'customer.tenure', 'phone.PhoneService', 'phone.MultipleLines',
       'internet.InternetService', 'internet.OnlineSecurity',
       'internet.OnlineBackup', 'internet.DeviceProtection',
       'internet.TechSupport', 'internet.StreamingTV',
       'internet.StreamingMovies', 'account.Contract',
       'account.PaperlessBilling', 'account.PaymentMethod',
       'account.Charges.Monthly', 'account.Charges.Total',
       'customer.SeniorCitizen'],
      dtype='object')

In [83]:
df.drop(['customerID','customer.SeniorCitizen','ChurnLabel'], axis=1, inplace=True)


In [89]:
df.rename({'SeniorLabel':'customer.SeniorCitizen'}, axis=1, inplace=True)


## Encoding

### Clasificación de variables

In [91]:
def clasificar_variable(col):
    if pd.api.types.is_numeric_dtype(col):
        return 'Cuantitativa continua' if col.nunique() > 10 else 'Cuantitativa discreta'
    elif col.nunique() == 2:
        return 'Cualitativa binaria'
    elif pd.api.types.is_object_dtype(col):
        return 'Cualitativa nominal u ordinal'
    else:
        return 'Tipo desconocido'

def generar_reporte(df, nombre_archivo='reporte_variables.md'):
    with open(nombre_archivo, 'w', encoding='utf-8') as f:
        f.write(f'# Clasificación de Variables\n\n')
        for col in df.columns:
            tipo = clasificar_variable(df[col])
            f.write(f"- **{col}**: {tipo}\n")

In [92]:
clasificacion = df.apply(clasificar_variable)
clasificacion

,0
Churn,Cualitativa binaria
customer.gender,Cualitativa binaria
customer.Partner,Cualitativa binaria
customer.Dependents,Cualitativa binaria
customer.tenure,Cuantitativa continua
phone.PhoneService,Cualitativa binaria
phone.MultipleLines,Cualitativa nominal u ordinal
internet.InternetService,Cualitativa nominal u ordinal
internet.OnlineSecurity,Cualitativa nominal u ordinal
internet.OnlineBackup,Cualitativa nominal u ordinal


In [93]:
def clasificar_simple_variable(col):
    if pd.api.types.is_numeric_dtype(col):
        return 'Cuantitativa'
    elif col.nunique() == 2:
        return 'Cualitativa'
    elif pd.api.types.is_object_dtype(col):
        return 'Cualitativa'
    else:
        return 'Desconocido'

In [94]:
Cualitativa=[]
Cuantitativa=[]


for col in df.columns:
    tipo = clasificar_simple_variable(df[col])
    if tipo == 'Cuantitativa':
        Cuantitativa.append(col)
    elif tipo == 'Cualitativa':
        Cualitativa.append(col)

print(Cualitativa)
print(Cuantitativa)


['Churn', 'customer.gender', 'customer.Partner', 'customer.Dependents', 'phone.PhoneService', 'phone.MultipleLines', 'internet.InternetService', 'internet.OnlineSecurity', 'internet.OnlineBackup', 'internet.DeviceProtection', 'internet.TechSupport', 'internet.StreamingTV', 'internet.StreamingMovies', 'account.Contract', 'account.PaperlessBilling', 'account.PaymentMethod', 'customer.SeniorCitizen']
['customer.tenure', 'account.Charges.Monthly', 'account.Charges.Total']


### Tabla de frecuencia para las variables cualitativas

In [95]:

for col in Cualitativa:
  print(f'Columna :{col}, Dato : {sorted(df[col].unique())}'	)

Columna :Churn, Dato : ['No', 'Yes']
Columna :customer.gender, Dato : ['Female', 'Male']
Columna :customer.Partner, Dato : ['No', 'Yes']
Columna :customer.Dependents, Dato : ['No', 'Yes']
Columna :phone.PhoneService, Dato : ['No', 'Yes']
Columna :phone.MultipleLines, Dato : ['No', 'No phone service', 'Yes']
Columna :internet.InternetService, Dato : ['DSL', 'Fiber optic', 'No']
Columna :internet.OnlineSecurity, Dato : ['No', 'No internet service', 'Yes']
Columna :internet.OnlineBackup, Dato : ['No', 'No internet service', 'Yes']
Columna :internet.DeviceProtection, Dato : ['No', 'No internet service', 'Yes']
Columna :internet.TechSupport, Dato : ['No', 'No internet service', 'Yes']
Columna :internet.StreamingTV, Dato : ['No', 'No internet service', 'Yes']
Columna :internet.StreamingMovies, Dato : ['No', 'No internet service', 'Yes']
Columna :account.Contract, Dato : ['Month-to-month', 'One year', 'Two year']
Columna :account.PaperlessBilling, Dato : ['No', 'Yes']
Columna :account.Payment

In [96]:
dis_frec_cualitativa_dict = {}
for col in Cualitativa:
    frecuencia = df[col].value_counts()
    porcentaje = df[col].value_counts(normalize=True).round(4) * 100
    dis_frec_cualitativa_dict[col] = pd.DataFrame({
        'Frecuencia': frecuencia,
        'Porcentaje %': porcentaje
    })


In [97]:
for col in Cualitativa:
 print(f'{dis_frec_cualitativa_dict[col]} \n')


       Frecuencia  Porcentaje %
Churn                          
No           5174         73.46
Yes          1869         26.54 

                 Frecuencia  Porcentaje %
customer.gender                          
Male                   3555         50.48
Female                 3488         49.52 

                  Frecuencia  Porcentaje %
customer.Partner                          
No                      3641          51.7
Yes                     3402          48.3 

                     Frecuencia  Porcentaje %
customer.Dependents                          
No                         4933         70.04
Yes                        2110         29.96 

                    Frecuencia  Porcentaje %
phone.PhoneService                          
Yes                       6361         90.32
No                         682          9.68 

                     Frecuencia  Porcentaje %
phone.MultipleLines                          
No                         3390         48.13
Yes                 

In [98]:
dis_frec_cualitativa_Churn_dict = {}
for col in Cualitativa:
    Cross = pd.crosstab(df['Churn'],df[col])
    dis_frec_cualitativa_Churn_dict[col] = Cross # Store the crosstab DataFrame directly

In [99]:
for col in Cualitativa:
 print(f'{dis_frec_cualitativa_Churn_dict[col]} \n')

Churn    No   Yes
Churn            
No     5174     0
Yes       0  1869 

customer.gender  Female  Male
Churn                        
No                 2549  2625
Yes                 939   930 

customer.Partner    No   Yes
Churn                       
No                2441  2733
Yes               1200   669 

customer.Dependents    No   Yes
Churn                          
No                   3390  1784
Yes                  1543   326 

phone.PhoneService   No   Yes
Churn                        
No                  512  4662
Yes                 170  1699 

phone.MultipleLines    No  No phone service   Yes
Churn                                            
No                   2541               512  2121
Yes                   849               170   850 

internet.InternetService   DSL  Fiber optic    No
Churn                                            
No                        1962         1799  1413
Yes                        459         1297   113 

internet.OnlineSecurity    No 

### Tabla de frecuencias para las variables cuantitativas

In [100]:
for col in Cuantitativa:
  print(f'Columna: {col}, Unicos: {df[col].unique()}'	)
  print(f'Columna: {col}, Valores:{df[col].value_counts()}\n')

Columna: customer.tenure, Unicos: [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42
  0]
Columna: customer.tenure, Valores:customer.tenure
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: count, Length: 73, dtype: int64

Columna: account.Charges.Monthly, Unicos: [65.6  59.9  73.9  ... 91.75 68.8  67.85]
Columna: account.Charges.Monthly, Valores:account.Charges.Monthly
20.05     61
19.85     45
19.95     44
19.90     44
19.70     43
          ..
113.45     1
117.50     1
116.15     1
62.55      1
116.20     1
Name: count, Length: 1585, dtype: int64

Columna: account.Charges.Total, Unicos: [ 593.3   542.4   280.85 ...  742.9  4627.65 3707.6 ]
Columna: account.Charges.Total, Valores:account.Charges.Total
20.20      11
0.00       11
19.75       9
19.90      

In [74]:
for col in Cuantitativa:
  print(f'Columna: {col}, Mínimo: {df[col].min()}'	)
  print(f'Columna: {col}, Máximo: {df[col].max()}')

Columna: customer.tenure, Mínimo: 0
Columna: customer.tenure, Máximo: 72
Columna: account.Charges.Monthly, Mínimo: 18.25
Columna: account.Charges.Monthly, Máximo: 118.75
Columna: account.Charges.Total, Mínimo: 0.0
Columna: account.Charges.Total, Máximo: 8684.8


In [75]:
import math
#n = len(df[col])  # Número total de datos individuales
#k = round(1 + math.log2(n))            # Regla de Sturges

import numpy as np
n=df.shape[0]
k=round(1+(10/3)*np.log10(n))

cuantitativa_dict = {}
for col in Cuantitativa:

  min=df[col].min()
  max=df[col].max()
  rango= max-min
  amplitud = math.ceil(rango / k)  # Redondeamos para cubrir el máximo
  labels = list(string.ascii_uppercase)
  print(f'Observaciones: {n}, Mínimo: {min}, Máximo: {max}, Amplitud: {amplitud}, Clases: {k}')
  inicio=min
  intervalos=[]
  etiquetas=[]
  intervalos.append(min)
  for i in range(k):
    fin = inicio + amplitud - 1  # Ajuste para cerrar el intervalo correctamente
    cuantitativa_dict[col] = fin
    print(labels[i], inicio,fin,i)
    inicio = fin + 1
    intervalos.append(fin)
    etiquetas.append(labels[i])
  cuantitativa_dict[col] = (intervalos,etiquetas[::-1])



NameError: name 'string' is not defined

In [ ]:
cuantitativa_dict


In [ ]:
dis_frec_cuantitativa_dict={}
for col in Cuantitativa:
  frec = pd.value_counts(pd.cut(x=df[col],
                                       bins=cuantitativa_dict[col][0],
                                       labels=cuantitativa_dict[col][1],
                                       include_lowest=True))
  porc = pd.value_counts(pd.cut(x=df[col],
                                       bins=cuantitativa_dict[col][0],
                                       labels=cuantitativa_dict[col][1],
                                       include_lowest=True),normalize=True).round(4) * 100
  dis_frec_cuantitativa_dict[col] = pd.DataFrame({
        'Frecuencia': frec,
        'Porcentaje %': porc
    })

In [ ]:
dis_frec_cuantitativa_dict['customer.tenure'].sort_index(ascending=False)

In [ ]:
import math
#n = len(df[col])  # Número total de datos individuales
#k = round(1 + math.log2(n))            # Regla de Sturges

import numpy as np
n=df.shape[0]
k=round(1+(10/3)*np.log10(n))
dis_frec_cuantitativa_dict_sturges={}
for col in Cuantitativa:
  frec = pd.value_counts(pd.cut(x=df[col],
                                       bins=k,
                                       include_lowest=True))
  porc = pd.value_counts(pd.cut(x=df[col],
                                       bins=k,
                                       include_lowest=True),normalize=True).round(4) * 100
  dis_frec_cuantitativa_dict_sturges[col] = pd.DataFrame({
        'Frecuencia': frec,
        'Porcentaje %': porc
    })



In [ ]:
dis_frec_cuantitativa_dict_sturges['customer.tenure'].sort_index(ascending=False)

In [ ]:
dis_frec_cuantitativa_dict_sturges

In [ ]:
import seaborn as sns

In [ ]:
for col in Cuantitativa:
 ax = sns.displot(data=df, x=col, kde=True)
 ax.fig.set_size_inches(12,6)
 ax.fig.suptitle(f'Distribución de {col}', fontsize=18)
 #ax.fig.set_xlabel(col, fontsize=14)
 ax

In [ ]:
ax = sns.displot(data=df, x='customer.tenure', kde=False)
ax.fig.set_size_inches(12,6)
ax.fig.suptitle('Distribución de tenure', fontsize=18)
ax.set_xlabel('Tenure', fontsize=14)

In [ ]:
import pandas as pd

# Carga del CSV desde GitHub
url = 'https://raw.githubusercontent.com/Tecindes/ChallengeONE__/refs/heads/main/reporte.csv'
df = pd.read_csv(url)

# Vista general y revisión de nulos
print(df.info())
print(df.isnull().sum())

In [ ]:
df.describe().to_markdown("resumen_datos.md")

In [ ]:
# One-Hot Encoding (ideal si no hay demasiadas categorías)
df_encoded = pd.get_dummies(df, drop_first=True)

In [ ]:
df_encoded